In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os
from PIL import Image

In [2]:
# Load data and create dataframe
phi = np.load("Phi_201snap.npy")
df = pd.DataFrame(index=np.arange(201))
my_dpi = 192

# Create convert function to use to convert output_array RGB values back to raw values
phimax, phimin = np.max(phi), np.min(phi)
ratio = (phimax-phimin)/255
def convertRGBToRaw(x):
    return ratio*x + phimin

for i in range(201):
    arr = phi[:,:,i]

    # Plot image (raw automatically converted to grayscale)
    plt.figure(figsize=(294/my_dpi, 165/my_dpi), dpi=my_dpi)
    plt.imshow(arr, cmap='gray')
    plt.axis("off")

    # Save as PNG then reopen. Then save as JPG's of varying quality (for loop). Convert back to np array.
    plt.savefig('2.1.png', bbox_inches='tight', pad_inches=0)
    plt.close()
    im = Image.open('2.1.png')
    im = im.convert('RGB') # need to convert from RGBA to RGB, otherwise saving as JPG doesn't work

    for q in [95, 75, 50, 25]:
        title = '2.1.' + str(q) + '.jpg'
        im.save(title, quality=q)
        im_jpg = Image.open(title)
        output_array = np.asarray(im_jpg)
    
        # Loop through output_array and replace RGB by raw in copy
        copy = np.zeros_like(arr)
        for row_index, row in enumerate(output_array):
            for col_index, col in enumerate(row):
                copy[row_index][col_index] = convertRGBToRaw(col[0])
    
        # Compute RMSE's and compression ratio and add to dataframe
        in_imsize = os.stat('2.1.png').st_size
        out_imsize = os.stat(title).st_size
        crcol = 'CR2.1.' + str(q)
        df.at[i, crcol] = in_imsize/out_imsize
        
        diff = copy - arr
        rmse = np.sqrt(np.mean(diff**2))
        rmsecol = 'RMSE2.1.' + str(q)
        df.at[i, rmsecol] = rmse
        
df.to_pickle("analysis2.1.pkl")
df

,CR2.1.95,RMSE2.1.95,CR2.1.75,RMSE2.1.75,CR2.1.50,RMSE2.1.50,CR2.1.25,RMSE2.1.25
0,2.225199,0.454821,4.148863,0.454839,5.226791,0.454862,5.761676,0.454662
1,2.238544,0.317059,4.128169,0.317107,5.130105,0.317030,6.043299,0.317010
2,2.120685,0.460035,3.985854,0.459934,4.829196,0.460129,5.577474,0.459528
3,2.046308,0.530838,3.891004,0.531039,4.719977,0.530802,5.527383,0.531217
4,2.123727,0.342132,3.999041,0.342164,4.878363,0.342246,5.725463,0.341761
...,...,...,...,...,...,...,...,...
196,1.858981,0.141536,3.425889,0.141600,4.147129,0.141432,4.917730,0.141780
197,1.834242,0.133754,3.333994,0.133860,4.082524,0.133731,4.792023,0.134080
198,1.823351,0.085889,3.298053,0.085908,4.052761,0.086110,4.685106,0.085741
199,1.836679,0.070893,3.361852,0.070967,4.126004,0.070906,4.781675,0.071113


In [13]:
df["RMSE2.1.95"].equals(df["RMSE2.1.25"]), df["RMSE2.1.95"].equals(df["RMSE2.1.50"]), df["RMSE2.1.95"].equals(df["RMSE2.1.75"])

(True, True, True)

In [33]:
# Load data and create dataframe
phi = np.load("Phi_201snap.npy")
df2 = pd.DataFrame(columns=['CR2.1', 'RMSE2.1'], index=np.arange(201))
my_dpi = 100

for i in range(201):
    arr = phi[:,:,i]

    # Plot image (raw automatically converted to grayscale)
    plt.figure(figsize=(294/my_dpi, 165/my_dpi), dpi=my_dpi)
    plt.imshow(arr, cmap='gray')
    plt.axis("off")

    # Save as PNG & JPG and open JPG again to convert to np array
    plt.savefig('analysis2.1.png', bbox_inches='tight', pad_inches=0)
    plt.savefig('analysis2.1.jpg', bbox_inches='tight', pad_inches=0)
    plt.close()
    im = Image.open('analysis2.1.jpg')
    output_array = np.asarray(im)

    # Create convert function to use to convert output_array back to raw values
    phimax, phimin = np.max(arr), np.min(arr)
    ratio = (phimax-phimin)/255
    def convertRGBToRaw(x):
        return ratio*x + phimin

    # Loop through output_array and replace RGB by raw in copy
    copy = arr.copy()
    for row_index, row in enumerate(output_array):
        for col_index, col in enumerate(row):
            copy[row_index][col_index] = convertRGBToRaw(col[0])

    # Compute RMSE's and compression ratio and add to dataframe
    in_imsize = os.stat('analysis2.1.png').st_size
    out_imsize = os.stat('analysis2.1.jpg').st_size
    df2.at[i, 'CR2.1'] = in_imsize/out_imsize
    
    diff = copy - arr
    rmse = np.sqrt(np.mean(diff**2))
    df2.at[i, 'RMSE2.1'] = rmse
# df.to_pickle("analysis2.1.pkl")
df2

,CR2.1,RMSE2.1
0,4.148863,0.0
1,4.128169,0.0
2,3.985854,0.0
3,3.891004,0.0
4,3.999041,0.0
...,...,...
196,3.425889,0.009345
197,3.333994,0.010284
198,3.298053,0.011375
199,3.361852,0.012638


In [5]:
# Computing single norms without for loop
i = 0

my_dpi = 192
phi = np.load("Phi_201snap.npy")
arr = phi[:,:,i] 
plt.figure(figsize=(294/my_dpi, 165/my_dpi), dpi=my_dpi)
plt.imshow(arr, cmap='gray')
plt.axis("off")
plt.savefig('analysis2.1.jpg', bbox_inches='tight', pad_inches=0)
plt.close()
im = Image.open('analysis2.1.jpg')
output_array = np.asarray(im)
output_array
copy = arr.copy()
phimax, phimin = np.max(arr), np.min(arr)
ratio = (phimax-phimin)/255
def convertRGBToRaw(x):
    return ratio*x + phimin
for i, row in enumerate(output_array):
    for j, col in enumerate(row):
        copy[i][j] = convertRGBToRaw(col[0])
norm = np.linalg.norm(copy - arr)
norm

1.3469364304465263e-08